In [1]:
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import numpy as np

### Reader

In [2]:
def readFile(file_path):
    image = Image.open(file_path)
    exif = image.getexif()
    if exif is None:
        return None
    
    exif_data = {}
    for tag_id, value in exif.items():
        tag = TAGS.get(tag_id, tag_id)
        if tag == 34682:
            metadata = value
    return metadata

In [3]:
testImg = '/Users/elias/Desktop/images_to_try/SEM_image_sample_Thermo_Fisher_Helios_G4_PFIB_CXe.tif'
# mdValueIni = readFile("/Users/elias/Desktop/NFDI Tomographiedaten/20200818_AlSi13 XRM tomo2/Images/SEM Image/SEM Image - SliceImage - 001.tif")
mdValueIni = readFile(testImg)
fileMetadataList = []

print(mdValueIni)

[User]
Date=03/17/2022
Time=11:05:00 AM
User=user
UserText=HeliosPFIB
UserTextUnicode=480065006C0069006F0073005000460049004200

[System]
Type=DualBeam
Dnumber=9952707
Software=14.5.1.432
BuildNr=432
Source=FEG
Column=Elstar
FinalLens=Elstar
Chamber=xT-SDB
Stage=110 x 110
Pump=TMP
ESEM=no
Aperture=AVA
Scan=PIA 3.0
Acq=PIA 3.0
EucWD=0.004
SystemType=Helios G4 PFIB CXe
DisplayWidth=0.518
DisplayHeight=0.324

[Beam]
HV=20000
Spot=2
StigmatorX=0
StigmatorY=0
BeamShiftX=-1.60064e-007
BeamShiftY=5.87256e-006
ScanRotation=0
ImageMode=Normal
FineStageBias=
Beam=EBeam
Scan=EScan

[EBeam]
Source=FEG
ColumnType=Elstar
FinalLens=Elstar
Acq=PIA 3.0
Aperture=AVA
ApertureDiameter=6.4e-005
HV=20000
HFW=0.0004144
VFW=0.000276267
WD=0.00794789
BeamCurrent=3.2e-009
TiltCorrectionIsOn=no
DynamicFocusIsOn=no
DynamicWDIsOn=
ScanRotation=0
LensMode=Field-Free
LensModeA=
ATubeVoltage=
UseCase=
SemOpticalMode=
ImageMode=Normal
SourceTiltX=0
SourceTiltY=0
StageX=-0.00333226
StageY=-0.0115452
StageZ=0.00800001
St

# Mapper (UNDER DEVELOPMENT)

Specs and data preparation ([details](https://github.com/kit-data-manager/Metadata-Schemas-for-Materials-Science/tree/main/SEM-FIB%20Tomography)):

* Acquisition metadata $\rightarrow$ get from .emxml file

* Dataset metadata $\rightarrow$ Some metadata is common to each image, identify which and extract only from first image, check against a few other images to ensure it really is the same?

* Image metadata $\rightarrow$ get from each image, just as it was for Zeiss


Therefore mapper should produce `n+2` JSON metadata documents, where `n` is the number of images in the dataset

In [21]:
# define a function which takes the metadata list and organizes it as a python dict

def formatMetadata(metadata):
    metadata_dict = {}
    current_header = None

    lines = metadata.strip().split('\n')

    for line in lines:
        line = line.strip()

        if line.startswith('[') and line.endswith(']'):
            current_header = line[1:-1]
        elif '=' in line:
            key, value = line.split('=', 1)
            split_key = key.split('.')
            last_variable = split_key[-1]
            formatted_last_variable = last_variable.lower()
            split_key[-1] = formatted_last_variable
            formatted_key = f'Images.SEM Image.SliceImage.{current_header}.' + '.'.join(split_key)
            value = value.strip()
            metadata_dict[formatted_key] = value

    return metadata_dict



image_data = formatMetadata(mdValueIni)
print(image_data)

{'Images.SEM Image.SliceImage.User.date': '03/17/2022', 'Images.SEM Image.SliceImage.User.time': '11:05:00 AM', 'Images.SEM Image.SliceImage.User.user': 'user', 'Images.SEM Image.SliceImage.User.usertext': 'HeliosPFIB', 'Images.SEM Image.SliceImage.User.usertextunicode': '480065006C0069006F0073005000460049004200', 'Images.SEM Image.SliceImage.System.type': 'DualBeam', 'Images.SEM Image.SliceImage.System.dnumber': '9952707', 'Images.SEM Image.SliceImage.System.software': '14.5.1.432', 'Images.SEM Image.SliceImage.System.buildnr': '432', 'Images.SEM Image.SliceImage.System.source': 'FEG', 'Images.SEM Image.SliceImage.System.column': 'Elstar', 'Images.SEM Image.SliceImage.System.finallens': 'Elstar', 'Images.SEM Image.SliceImage.System.chamber': 'xT-SDB', 'Images.SEM Image.SliceImage.System.stage': '110 x 110', 'Images.SEM Image.SliceImage.System.pump': 'TMP', 'Images.SEM Image.SliceImage.System.esem': 'no', 'Images.SEM Image.SliceImage.System.aperture': 'AVA', 'Images.SEM Image.SliceImag

In [33]:
import json

def extractImageMappings(json_file):
    with open(json_file) as f:
        mappings = json.load(f)
    image_mappings = mappings.get('image', {})
    return image_mappings

mapFile = "/Users/elias/Desktop/PP13_Mapping/pp13-mapper/schemas/sem_fib_nested_schema_map.json"
image_mappings = extractImageMappings(mapFile)

# print(image_mappings)

def extractImageData(image_data, image_mappings):
    extracted_data = {}

    for key, value in image_data.items():
        if key in image_mappings.values():
            extracted_data[key] = value

    return extracted_data


image_metadata = extractImageData(image_data, image_mappings)

In [35]:
def headerMapping(image_metadata, image_mappings):
    mapped_metadata = {}

    for desired_var, current_var in image_mappings.items():
        if current_var in image_metadata:
            mapped_metadata[desired_var] = image_metadata[current_var]

    return mapped_metadata



headerMapping(image_metadata, image_mappings)

{'acquisition.dataset.entry.images.entry.creationTime': '17.03.2022 11:05:00',
 'acquisition.dataset.entry.images.entry.stage.workingDistance.value': '0.00794789',
 'acquisition.dataset.entry.images.entry.stage.stageX.value': '-0.00333242',
 'acquisition.dataset.entry.images.entry.stage.stageY.value': '-0.0115393',
 'acquisition.dataset.entry.images.entry.stage.stageZ.value': '0.00800001',
 'acquisition.dataset.entry.images.entry.stage.stageR.value': '1.49703',
 'acquisition.dataset.entry.images.entry.stage.stageTa.value': '1.95126e-006',
 'acquisition.dataset.entry.images.entry.stage.stageTb.value': '0',
 'acquisition.dataset.entry.images.entry.stage.specTilt.value': '',
 'acquisition.dataset.entry.images.entry.stage.activeStage': 'Bulk',
 'acquisition.dataset.entry.images.entry.vacuum.chamberPressure.value': '8.97e-005',
 'acquisition.dataset.entry.images.entry.vacuum.gas': '',
 'acquisition.dataset.entry.images.entry.vacuum.userMode': 'High vacuum',
 'acquisition.dataset.entry.image